![Renode](https://dl.antmicro.com/projects/renode/renode.png)
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/antmicro/tensorflow-arduino-examples/blob/master/examples/micro-speech/micro_speech.ipynb"><img src="https://raw.githubusercontent.com/antmicro/tensorflow-arduino-examples/master/examples/.static/view-in-colab.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/antmicro/tensorflow-arduino-examples/blob/master/examples/micro-speech/micro_speech.ipynb"><img src="https://raw.githubusercontent.com/antmicro/tensorflow-arduino-examples/master/examples/.static/view-ipynb.png" />View ipynb on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/antmicro/tensorflow-arduino-examples/blob/master/examples/micro-speech/micro_speech.py"><img src="https://raw.githubusercontent.com/antmicro/tensorflow-arduino-examples/master/examples/.static/view-source.png" />View Python source on GitHub</a>
  </td>
</table>

## Install requirements

In [ ]:
!pip install -q ffmpeg-python pyaudioconvert git+https://github.com/antmicro/pyrenode.git git+https://github.com/antmicro/renode-colab-tools.git
!apt install -q xxd sox
!mkdir -p renode && cd renode && wget https://dl.antmicro.com/projects/renode/builds/renode-latest.linux-portable.tar.gz && tar -xzf renode-latest.linux-portable.tar.gz --strip 1
!pip install -q -r renode/tests/requirements.txt
!wget -O binary_yes https://dl.antmicro.com/projects/renode/audio_yes_1s.s16le.pcm-s_32000-b69f5518615516f80ae0082fe9b5a5d29ffebce8
!wget -O binary_no https://dl.antmicro.com/projects/renode/audio_no_1s.s16le.pcm-s_32000-f36b60d425c9d5414a38658dbd096313d82d28c5

import os
from renode_colab_tools import audio, metrics
os.environ['PATH'] = os.getcwd()+"/renode:"+os.environ['PATH']
os.environ['TENSORFLOW_PATH'] = os.getcwd()+"/tensorflow-arduino-examples/tensorflow"

In [ ]:
!mkdir -p binaries/micro_speech && cd binaries/micro_speech && wget https://github.com/antmicro/tensorflow-arduino-examples-binaries/raw/master/micro_speech/micro_speech.ino.elf # fetch prebuilt binaries

## Get audio

In [ ]:
audio.audio_options()

## Run a micro-speech example in Renode
The command may not be recognized correctly due to https://github.com/tensorflow/tensorflow/pull/45878

In [ ]:
import time
from pyrenode import *
shutdown_renode()
connect_renode() # this sets up a log file, and clears the simulation (just in case)

tell_renode('using sysbus')
tell_renode('mach create')
tell_renode('machine LoadPlatformDescription @platforms/boards/arduino_nano_33_ble.repl')
tell_renode('sysbus LoadELF @binaries/micro_speech/micro_speech.ino.elf')

tell_renode('logLevel 3')
tell_renode('pdm SetInputFile @audio_bin')
tell_renode('uart0 CreateFileBackend @uart.dump true')
tell_renode('machine EnableProfiler @metrics.dump')
tell_renode('s')
while not os.path.exists('renode/uart.dump'):
  time.sleep(1) #waits for creating uart.dump
!timeout 60 tail -f renode/uart.dump | sed '/^.*Heard .*$/ q' | cut -c 2-
tell_renode('q')
expect_cli('Renode is quitting')
time.sleep(1) #wait not to kill Renode forcefully
shutdown_renode()

## Renode metrics analysis

In [ ]:
from renode.tools.metrics_analyzer.metrics_parser import MetricsParser
metrics.init_notebook_mode(connected=False)
parser = MetricsParser('renode/metrics.dump')

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_executed_instructions(parser)

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_memory_access(parser)

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_peripheral_access(parser)

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_exceptions(parser)